## A. Configuraciones Generales.

In [ ]:
#1. Librerías.
%run "./librerias.ipynb"

In [ ]:
#2. Constantes.
#a. Constantes generales.
%run "./constantes.ipynb"

#b. Constantes a definir por el usuario.
#i. Dataset post Future Engineering del modelo.
dataset_input = dataset_file_fe_12_3
#ii. Mes de entrenamiento.
mes_train = mes_train_ult_anio
#iii. Ruta del modelo guardado post-optimización de hiperparámetros.
modelo =  "{}".format(modelos_path) + 'rf_1_3.sav'
#iv. Dataset donde exportar los resultados obtenidos.
dataset_output = path_output_m4

In [ ]:
#3. Funciones.
%run "./funciones.ipynb"

In [ ]:
#4. Lectura de dataframes post-feature engineering (.csv o .parquet), y modelos ya optimizados previamente (.txt).
#i. Modelo LGBM propio con 3 meses para calculo de slope y valor actual vs promedio histórico. 
data = pd.read_parquet(dataset_input)

In [ ]:
#5. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)

In [ ]:
#7. Dividimos entre conjuntos de datos.
#a. Train.
X = data[data['foto_mes'].isin(mes_train)]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])

In [ ]:
#4. Importo el modelo.
#i. Lo importo.
model_rf = pickle.load(open(modelo, 'rb'))
#ii. Extraigo los mejores parámetros del modelo.
best_params = model_rf.get_params()

In [ ]:
#9. Dataframe con predicciones finales.
stacking_predictions = pd.DataFrame(columns=['numero_de_cliente', 'foto_mes', 'probabilidad'])

## B. Entrenamiento de Modelo Individual, y predicción de las folds restantes.

In [ ]:
#1. Entrenamos y predecimos.
#i. Se crean las 5 folds estratificadas por mes.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=semillas[0])

#ii. Se recorre cada fold, dividiendo entre 4 para train y 1 para validación.
for fold, (train_index, val_index) in enumerate(skf.split(X, X['foto_mes'])):
    print(f"Fold {fold + 1}")
    
    #a. Se dividen los datos en conjuntos de entrenamiento y validación para este fold.
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    #b. Entrenamiento del modelo en los datos del fold actual usando los parámetros del modelo original.
    model_rf = RandomForestClassifier(
    **best_params # Usa los mejores hiperparámetros obtenidos de Optuna
)
    
    model_rf.fit(X_train_fold, y_train_fold)
    
    #d. Se predice el conjunto de validación para el fold actual.
    val_predictions = model_rf.predict_proba(X_val_fold)[:, 1]
    
    #e. Se crea un DataFrame temporal para almacenar las predicciones2.
    fold_predictions = pd.DataFrame({
        'numero_de_cliente': X.iloc[val_index]['numero_de_cliente'].values,
        'foto_mes': X.iloc[val_index]['foto_mes'].values,
        'probabilidad': val_predictions
    })
    
    #f. Se agregan las predicciones del fold al DataFrame final.
    stacking_predictions = pd.concat([stacking_predictions, fold_predictions], ignore_index=True)

In [ ]:
#2. Renombramos.
stacking_predictions.rename({"probabilidad":"probabilidad_modelo4"},axis=1,inplace=True)

In [ ]:
#3. Exportamos.
stacking_predictions.to_parquet(dataset_output)